1. PARSE XML FILES
2. TURN INTO TREE
3. IF FAILS, PARSE AS MANY OF THE ENTRIES AS POSSIBLE

In [41]:
import traceback
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import shutil
import os

import git

repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)

srcpaths=(
    "first_run_xmls",
)

xmls = []
for d in srcpaths:
    l = list(Path(d).glob("*.xml"))
    l = [p for p in l if ".repair." not in p.name]
    xmls += l
print(len(xmls), "XML files found,", len(set([f.name for f in xmls])), "unique. duplicates =", [f for f in xmls if [g.name for g in xmls].count(f.name) > 1])

234 XML files found, 185 unique. duplicates = [PosixPath('/run/media/benjis/MAGI/basilisk-backup/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-configuration-JSONConfigurationWriteFuzzer.xml'), PosixPath('/run/media/benjis/MAGI/basilisk-backup/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-kryo-DeserializeNumbersFuzzer.xml'), PosixPath('/run/media/benjis/MAGI/basilisk-backup/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-httpcomponents-client-HttpFuzzer.xml'), PosixPath('/run/media/benjis/MAGI/basilisk-backup/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-guava-MediaTypeFuzzer.xml'), PosixPath('/run/media/benjis/MAGI/basilisk-backup/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-protobuf-java-ProtobufFuzzer.xml'), PosixPath('/run/media/benjis/MAGI/basilisk-bac

In [36]:
dstdir=Path("first_run_xmls_postprocessed")

In [37]:
if dstdir.exists():
    shutil.rmtree(dstdir)
dstdir.mkdir(exist_ok=True)

In [40]:
import tqdm as tqdm
# from lxml import etree
from lxml import etree as ET2
parser = ET2.XMLParser(recover=True)

successes_1 = []
failed_1 = []
with tqdm.tqdm(xmls) as pbar:
    for fpath in pbar:
        try:
            tree = ET2.parse(fpath, parser=parser)
            successes_1.append(fpath)
            # shutil.copyfile(fpath, dstdir/fpath.name)
            tree.write(dstdir/fpath.name, pretty_print=True)
        except ET2.XMLSyntaxError as ex:
            failed_1.append(fpath)
        except ET.ParseError as ex:
            failed_1.append(fpath)
        pbar.set_postfix({
            "success": len(successes_1),
            "failed": len(failed_1),
        })

print("ROUND 1:", len(failed_1), "files failed parsing")
(dstdir/"1_repair_success_1.txt").write_text("\n".join(map(str, successes_1)))

100%|██████████| 56/56 [03:15<00:00,  3.49s/it, success=56, failed=0]

ROUND 1: 0 files failed parsing


3617

In [ ]:
import tqdm as tqdm

fuzzer_start = re.compile(r'''<call[^>]*method="[^("]+.fuzzerTestOneInput\([^)]+\)"[^>]*''')

from xml.dom import minidom
from xml.parsers.expat import ExpatError

def recover_functions_simple(fpath, dstdir):
    repair_path = dstdir/(str(fpath.name) + ".repair.xml")
    if repair_path.exists():
        return repair_path
    
#     with open(fpath) as f:
#         num_lines = sum(1 for line in tqdm.tqdm(f, desc=f"count lines ({fpath.name})"))
    last_fuzzer_start = None
    with open(fpath) as f:
        i = 0
        it = iter(f)
        failed_functions = 0
        all_functions = 0
        with tqdm.tqdm(it, total=None, desc=f"chop final fuzzer target call ({fpath.name})") as pbar:
            it = pbar
            fuzz_functions = []
            current_fuzz_function = None
            for i, line in enumerate(it):
                m = fuzzer_start.search(line)
                if m is not None:
#                     print("MATCH", m, m.start(), m.span())
                    j = m.start()
                    last_fuzzer_start = (i, j)
#     print("LFS", last_fuzzer_start)
    if last_fuzzer_start is None:
        print("ERROR: last_fuzzer_start is None", fpath)
        return None
    
    with open(fpath) as f, open(repair_path, "w") as outf:
        it = iter(f)
        with tqdm.tqdm(it, total=last_fuzzer_start[0], desc=f"write file ({fpath.name})") as pbar:
            it = pbar
            for i, line in enumerate(it):
#                 print(i)
                if i == last_fuzzer_start[0]:
#                     print("LINE:")
#                     print(line)
                    line_chopped = line[:last_fuzzer_start[1]]
#                     print("CHOPPED:")
#                     print(line_chopped)
                    outf.write(line_chopped)
                    break
                else:
                    outf.write(line)
            outf.write("</trace>")
        
    return repair_path

# xml_file = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
# recover_functions_simple(xml_file, Path("postprocessed_xmls_debug"))

In [ ]:
# fpath = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
# repair_path = recover_functions_simple(fpath, Path("postprocessed_xmls_debug"))
# print(fpath, "->", repair_path)
# it = ET.iterparse(repair_path)
# for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
#     pass

In [ ]:
# successes_1 = list(map(Path, (dstdir/"1_repair_success_1.txt").read_text().split("\n")))
# failed_1 = [fpath for fpath in xmls if fpath.name not in [s.name for s in successes_1]]
# len(successes_1), len(failed_1)

In [ ]:
import tqdm
successes_2 = []
failed_2 = []
with tqdm.tqdm(failed_1, position=1, desc="round 2") as pbar:
    for fpath in pbar:
        repair_path = recover_functions_simple(fpath, dstdir)
        if repair_path is None:
            print("ERROR", fpath)
            continue
        try:
            it = ET.iterparse(repair_path)
            for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
                pass
            successes_2.append(repair_path)
            # shutil.copyfile(repair_path, dstdir/repair_path.name)
        except ET.ParseError as ex:
            print("exception", type(ex), ex, "parsing", fpath, repair_path)
            failed_2.append(repair_path)
        pbar.set_postfix({
            "success": len(successes_2),
            "failed": len(failed_2),
        })

print("ROUND 2:", len(failed_2), "files failed parsing")
(dstdir/"1_repair_success_2.txt").write_text("\n".join(map(str, successes_2)))

In [34]:
# lxml XMLParser(recover=True) is needed for some gnarly examples.
xml = "/home/benjis/code/bug-benchmarks/oss-fuzz/traces-10m/logs-xmls/trace-apache-commons-configuration-JSONConfigurationReadFuzzer.xml"

from lxml import etree as ET2
parser = ET2.XMLParser(recover=True)
it = ET2.parse(xml, parser=parser)

try:
    it = ET2.parse(xml)
except ET2.XMLSyntaxError:
    print("Got expected error from lxml normal parsing")

import xml.etree.ElementTree as ET
try:
    # from xml.etree import XMLSyntaxError
    it = ET.parse(xml)
except ET.ParseError:
    print("Got expected error from ElementTree normal parsing")

Got expected error from lxml normal parsing
Got expected error from ElementTree normal parsing


In [12]:
import tqdm as tqdm
import re
from pathlib import Path

fuzzer_start = re.compile(r'''<call[^>]*method="[^("]+.fuzzerTestOneInput\([^)]+\)"[^>]*''')
fuzzer_almost_end = re.compile(r'''<tracepoint[^>]+>''')
method_re = re.compile(r'''method="[^("]+.fuzzerTestOneInput\([^)]+\)"''')
exit_re = re.compile(r'''type="exit"''')
fuzzer_real_end = re.compile(r'''</call>''')


from xml.dom import minidom
from xml.parsers.expat import ExpatError

def prettify(rough_string):
    """Return a pretty-printed XML string for the Element.
    """
    reparsed = minidom.parseString(rough_string)
    
    return reparsed.toprettyxml(indent="  ")

def recover_functions(fpath, dstdir):
    with open(fpath) as f:
        num_lines = sum(1 for line in tqdm.tqdm(f, desc=f"count lines ({fpath.name})"))
    with open(fpath) as f:
        it = iter(f)
        repair_path = dstdir/(str(fpath.name) + ".repair.xml")
        failed_functions = 0
        all_functions = 0
        with open(repair_path, "w") as outf:
            with tqdm.tqdm(it, total=num_lines, desc=f"deconstruct into fuzzer target calls ({fpath.name})") as pbar:
                it = iter(pbar)
                fuzz_functions = []
                current_fuzz_function = None
                try:
                    outf.write(next(it))
                    while True:
                        line = next(it)
                        if fuzzer_start.search(line):
                            print("start at", line)
                            # start fuzzed function
                            current_fuzz_function = []
                        if current_fuzz_function is not None:
                            current_fuzz_function.append(line)
                        m = fuzzer_almost_end.search(line)
                        if m:
                            tag = m.group(0)
                            if method_re.search(tag) and exit_re.search(tag):
                                print("end at", line)
                                while True:
                                    line = next(it)
                                    print("search end", line)
                                    current_fuzz_function.append(line)
                                    if fuzzer_real_end.search(line):
                                        # cap off fuzzed function
                                        print("end", line)
                                        all_functions += 1
                                        try:
                                            func_xml = "".join(current_fuzz_function)
                                            # ET.fromstring(func_xml)
                                            print(func_xml)
                                            func_xml = "".join(prettify(func_xml).splitlines(keepends=True)[1:])
                                            outf.write(func_xml + "\n")
                                        except (ExpatError, ET.ParseError):
                                            failed_functions += 1
                                        pbar.set_postfix({"all": all_functions, "failed": failed_functions})
                                        break
                except StopIteration:
                    pass
            outf.write("</trace>")

        return repair_path

recover_functions(Path("/home/benjis/code/bug-benchmarks/oss-fuzz/traces-10m/logs-xmls/trace-okhttp-LoggingInterceptorFuzzer.xml"), Path("/home/benjis/code/bug-benchmarks/oss-fuzz/processed_xml_debug"))

count lines (trace-okhttp-LoggingInterceptorFuzzer.xml): 30263it [00:00, 572310.72it/s]
deconstruct into fuzzer target calls (trace-okhttp-LoggingInterceptorFuzzer.xml): 100%|██████████| 30263/30263 [00:00<00:00, 213367.68it/s]

start at <call method="ossfuzz.LoggingInterceptorFuzzer.fuzzerTestOneInput(com.code_intelligence.jazzer.api.FuzzedDataProvider)" location="ossfuzz.LoggingInterceptorFuzzer:74" ordinal="0">



PosixPath('/home/benjis/code/bug-benchmarks/oss-fuzz/processed_xml_debug/trace-okhttp-LoggingInterceptorFuzzer.xml.repair.xml')

In [ ]:

import tqdm
successes_3 = []
failed_3 = []
with tqdm.tqdm(failed_2, position=1, desc="round 3") as pbar:
    for fpath in pbar:
        repair_path = recover_functions(fpath, dstdir)
        try:
            it = ET.iterparse(repair_path)
            for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
                pass
            successes_3.append(repair_path)
        except ET.ParseError as ex:
            print("exception", type(ex), ex, "parsing", fpath)
            failed_3.append(repair_path)
        pbar.set_postfix({
            "success": len(successes_3),
            "failed": len(failed_3),
        })

print("ROUND 3:", len(failed_3), "files failed parsing")
(dstdir/"1_repair_success_3.txt").write_text("\n".join(map(str, successes_3)))
failed_3

In [ ]:
all_files = successes_1 + successes_2 + successes_3

In [ ]:
with open(dstdir/"1_repair_success_all.txt", "w") as f:
    f.write("\n".join(map(str, all_files)))